In [1]:
import tensorflow as tf
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
tf.disable_v2_behavior()
tf.__version__

Instructions for updating:
non-resource variables are not supported in the long term


'2.2.0'

In [3]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
# one-hot encoding : [0,0,1]은 2 [0,1,0]은 1 [1,0,0]은 0을 의미함 
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

# X_data의 instance, columns
X = tf.placeholder("float", [None, 4])
# Y_data의 instance, columns
Y = tf.placeholder("float", [None, 3])
# Y_data의 label의 개수 
nb_classes = 3

# X의 columns 개수, Y로 나가는 값(class 개수와 같음)
W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
# class 개수와 같음
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))

# GradientDescentOptimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
            _, cost_val = sess.run([optimizer, cost], feed_dict={X: x_data, Y: y_data})

            if step % 200 == 0:
                print(step, cost_val)

    print('--------------')
    # Testing & One-hot encoding
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, 1)))

    print('--------------')
    b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b, 1)))

    print('--------------')
    c = sess.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.argmax(c, 1)))

    print('--------------')
    all = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.argmax(all, 1)))


0 4.6630363
200 0.5887856
400 0.4749389
600 0.3770786
800 0.2849663
1000 0.23799756
1200 0.21523324
1400 0.19638564
1600 0.18050188
1800 0.16693038
2000 0.15520263
--------------
[[5.1263147e-03 9.9486583e-01 7.7984269e-06]] [1]
--------------
[[0.8610865  0.12997028 0.00894323]] [0]
--------------
[[1.07264615e-08 3.28499358e-04 9.99671459e-01]] [2]
--------------
[[5.1263142e-03 9.9486583e-01 7.7984341e-06]
 [8.6108649e-01 1.2997028e-01 8.9432281e-03]
 [1.0726461e-08 3.2849936e-04 9.9967146e-01]] [1 0 2]


In [6]:
import numpy as np

# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
# instance는 전체, 처음~맨뒤column 제외 값
x_data = xy[:, 0:-1]
# instance는 전체, 맨뒤 column 값
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)
print(x_data)
print(y_data)

'''
(101, 16) (101, 1)
'''

# y_data의 label 개수(여기 예제에서는 동물 종류의 개수)
# y_data.unique() 와 동일
nb_classes = 7  # 0 ~ 6

# X_data의 instance, columns
X = tf.placeholder(tf.float32, [None, 16])
# Y_data의 instance, column
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

# y_data는 0~6까지의 숫자(class=7)인데, 이를 one_hot으로 바꿔줌
# rank N이면 output이 N+1이 된다.
# [[0],[3]]  ===> [[[10000000]]] 3차원으로 올라감
Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("one_hot:", Y_one_hot)
# 자동으로 reshpae을 해준다. [[1000000],[0001000]] 
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape one_hot:", Y_one_hot)

'''
one_hot: Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape:0", shape=(?, 7), dtype=float32)
'''
# X_data의 columns, y로 나가는 값의 개수
W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
# y로 나가는 값의 개수
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
# cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis=1)) 과 동일한 표현임
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                 labels=tf.stop_gradient([Y_one_hot])))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# softmax에서 가장 큰 값을 찾음(예측)
prediction = tf.argmax(hypothesis, 1)
# 정답인지 판단
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
# 정답률 판단
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        _, cost_val, acc_val = sess.run([optimizer, cost, accuracy], feed_dict={X: x_data, Y: y_data})
                                        
        if step % 100 == 0:
            print("Step: {:5}\tCost: {:.3f}\tAcc: {:.2%}".format(step, cost_val, acc_val))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))


(101, 16) (101, 1)
[[1. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 1. 0. 1.]
 [0. 0. 1. ... 1. 0. 0.]
 ...
 [1. 0. 0. ... 1. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 1. 0. 0.]]
[[0.]
 [0.]
 [3.]
 [0.]
 [0.]
 [0.]
 [0.]
 [3.]
 [3.]
 [0.]
 [0.]
 [1.]
 [3.]
 [6.]
 [6.]
 [6.]
 [1.]
 [0.]
 [3.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [5.]
 [4.]
 [4.]
 [0.]
 [0.]
 [0.]
 [5.]
 [0.]
 [0.]
 [1.]
 [3.]
 [0.]
 [0.]
 [1.]
 [3.]
 [5.]
 [5.]
 [1.]
 [5.]
 [1.]
 [0.]
 [0.]
 [6.]
 [0.]
 [0.]
 [0.]
 [0.]
 [5.]
 [4.]
 [6.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [3.]
 [3.]
 [2.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [6.]
 [3.]
 [0.]
 [0.]
 [2.]
 [6.]
 [1.]
 [1.]
 [2.]
 [6.]
 [3.]
 [1.]
 [0.]
 [6.]
 [3.]
 [1.]
 [5.]
 [4.]
 [2.]
 [2.]
 [3.]
 [0.]
 [0.]
 [1.]
 [0.]
 [5.]
 [0.]
 [6.]
 [1.]]
one_hot: Tensor("one_hot_1:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape_1:0", shape=(?, 7), dtype=float32)
Step:     0	Cost: 8.165	Acc: 20.79%
Step:   100	Cost: 0.734	Acc: 77.23%
Step:   200	Cos